In [4]:
import transformers
import evaluate
import torch
import operator
import ast
import json
import re
import sys

import numpy as np
import pandas as pd
import torch.nn as nn

from multiprocessing import cpu_count
from evaluate import load
from nusacrowd import NusantaraConfigHelper
from torch.utils.data import DataLoader
from datetime import datetime
from huggingface_hub import notebook_login
from tqdm import tqdm

from datasets import (
    load_dataset, 
    load_from_disk,
    Dataset,
    DatasetDict
)
from transformers import (
    BigBirdTokenizerFast,
    BigBirdForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    BertForSequenceClassification,
    BertForQuestionAnswering,
    AutoModel, 
    BertTokenizerFast,
    AutoTokenizer, 
    AutoModel, 
    BertTokenizer, 
    BertForPreTraining,
    AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering,
    XLMRobertaTokenizer
)

In [37]:
def question_type(data, nama):
    num_apa = 0
    num_dimana = 0
    num_kapan = 0
    num_siapa = 0
    num_bagaimana = 0
    num_kenapa = 0
    num_berapa = 0
    num_others = 0
    
    for i in data['question']:
        current_question = i.split()
        
        if 'Apa' in current_question: num_apa += 1
        elif 'Apakah' in current_question:num_apa += 1
        elif 'apa' in current_question:num_apa += 1
        elif 'apakah' in current_question:num_apa += 1
        
        elif 'Dimana' in current_question: num_dimana += 1
        elif 'dimana' in current_question: num_dimana += 1
        elif 'mana' in current_question: num_dimana += 1
        
        elif 'Kapan' in current_question: num_kapan += 1
        elif 'kapan' in current_question: num_kapan += 1
        
        elif 'Siapa' in current_question: num_siapa += 1
        elif 'siapa' in current_question: num_siapa += 1
        
        elif 'Bagaimana' in current_question: num_bagaimana += 1
        elif 'bagaimana' in current_question: num_bagaimana += 1

        elif 'Mengapa' in current_question: num_kenapa += 1
        elif 'Kenapa' in current_question: num_kenapa += 1
        elif 'mengapa' in current_question: num_kenapa += 1
        elif 'kenapa' in current_question: num_kenapa += 1
        
        elif 'Berapa' in current_question: num_berapa += 1
        elif 'Berapakah' in current_question: num_berapa += 1
        elif 'berapa' in current_question: num_berapa += 1
        elif 'berapakah' in current_question: num_berapa += 1
        
        else: num_others += 1
        
    denominator = len(data['question'])
    num_apa_percentage = num_apa / denominator
    num_dimana_percentage = num_dimana / denominator
    num_kapan_percentage = num_kapan / denominator
    num_siapa_percentage = num_siapa / denominator
    num_kenapa_percentage = num_kenapa / denominator
    num_bagaimana_percentage = num_bagaimana / denominator
    num_berapa_percentage = num_berapa / denominator
    num_others_percentage = num_others / denominator
    
    print(f"{str(nama)} memiliki banyak pertanyaan APA sekitar: {num_apa}")
    print(f"{str(nama)} memiliki banyak pertanyaan DIMANA sekitar: {num_dimana}")
    print(f"{str(nama)} memiliki banyak pertanyaan KAPAN sekitar: {num_kapan}")
    print(f"{str(nama)} memiliki banyak pertanyaan SIAPA sekitar: {num_siapa}")
    print(f"{str(nama)} memiliki banyak pertanyaan KENAPA sekitar: {num_kenapa}")
    print(f"{str(nama)} memiliki banyak pertanyaan BAGAIMANA sekitar: {num_bagaimana}")
    print(f"{str(nama)} memiliki banyak pertanyaan BERAPA sekitar: {num_berapa}")
    print(f"{str(nama)} memiliki banyak pertanyaan LAINNYA sekitar: {num_others}")
    print()
        
    print(f"{str(nama)} memiliki banyak pertanyaan APA sekitar: {num_apa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan DIMANA sekitar: {num_dimana_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan KAPAN sekitar: {num_kapan_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan SIAPA sekitar: {num_siapa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan KENAPA sekitar: {num_kenapa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan BAGAIMANA sekitar: {num_bagaimana_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan BERAPA sekitar: {num_berapa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan LAINNYA sekitar: {num_others_percentage * 100} %")

In [83]:
def pick_a_row(data, question_type, number):
    
    cols = ['context', 'question', 'answer']
    target_question_type_df = pd.DataFrame(columns=cols)

    for i in range(len(data)):

        current_question = data['question'][i].split()
        
        if question_type == 'lainnya':
            if 'Apa' in current_question: pass
            elif 'Apakah' in current_question: pass
            elif 'apa' in current_question: pass
            elif 'apakah' in current_question: pass
            elif 'Dimana' in current_question: pass
            elif 'dimana' in current_question: pass
            elif 'mana' in current_question: pass
            elif 'Kapan' in current_question: pass
            elif 'kapan' in current_question: pass
            elif 'Siapa' in current_question: pass
            elif 'siapa' in current_question: pass
            elif 'Bagaimana' in current_question: pass
            elif 'bagaimana' in current_question: pass
            elif 'Mengapa' in current_question: pass
            elif 'Kenapa' in current_question: pass
            elif 'mengapa' in current_question: pass
            elif 'kenapa' in current_question: pass
            elif 'Berapa' in current_question: pass
            elif 'Berapakah' in current_question: pass
            elif 'berapa' in current_question: pass
            elif 'berapakah' in current_question: pass

            else: 
                target_question_type_df = target_question_type_df.append({
                                             'context': data["context"][i], 
                                             'question': data['question'][i], 
                                             'answer': data['answer'][i]}, 
                                                                   ignore_index=True)
            continue

        for j in question_type:
            
            if j in current_question: 
                target_question_type_df = target_question_type_df.append({
                                             'context': data["context"][i], 
                                             'question': data['question'][i], 
                                             'answer': data['answer'][i]}, 
                                                                   ignore_index=True)

    target_question_type_df = target_question_type_df.sample(n=number)
    return target_question_type_df

# Bagian IDK-MRC

In [73]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'idk_mrc' in x.dataset_name)[0].load_dataset()

df_test = pd.DataFrame(data_qas_id['test'])

cols = ['context', 'question', 'answer']
new_df_test = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_test['context']))):
    for j in df_test["qas"][i]:
        if len(j['answers']) != 0:
            new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                                'question': j['question'], 
                                                'answer': j['answers'][0]['text'] 
                                                           }, 
                                                           ignore_index=True)
        else:
            new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                                'question': j['question'], 
                                                'answer': "-", 
                                                           }, 
                                                           ignore_index=True)

test_dataset = Dataset.from_dict(new_df_test)

data_qas_id_idk_mrc = DatasetDict({"test": test_dataset})
data_qas_id_idk_mrc = pd.DataFrame(data_qas_id_idk_mrc['test'])
data_qas_id_idk_mrc

  0%|          | 0/3 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 378/378 [00:01<00:00, 197.89it/s]


,context,question,answer
0,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer mikro mulai dikembangkan ?,1959
1,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan komputer mikro mulai ditinggalkan?,-
2,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer mikro mulai dikembangkan Amerik...,-
3,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer makro mulai dikembangkan?,-
4,"Patronim, atau patronimik, adalah sebuah kompo...",Apakah pengertian matronimik?,Komomene sebuah nama yang berdasarkan pada nam...
...,...,...,...
839,Umur dewasa dari drummer ganda adalah sekitar ...,Bagaimana cara jangkrik betina hidup?,-
840,Kota ini terletak di sebuah lembah sungai yang...,"Menurut Biro Sensus Amerika Serikat, berapa lu...","Menurut Biro Sensus Amerika Serikat, kota ini ..."
841,Kota ini terletak di sebuah lembah sungai yang...,"Menurut Biro Sensus Amerika Serikat, berapa vo...",-
842,"Tanpa beasiswa, Ogilvy tidak bisa kuliah di Fe...",Apa alasan Ogilvy tidak bisa kuliah di Fettes ...,Ogilvy tidak bisa kuliah di Fettes atau Oxford...


In [43]:
question_type(data=data_qas_id_idk_mrc, nama="IDK-MRC Test")

# Target: 100 anotasi -> 100/8 = 12.5
# APA sebanyak 14
# DIMANA sebanyak 13
# KAPAN sebanyak 14
# SIAPA sebanyak 13
# KENAPA sebanyak 8
# BAGAIMANA sebanyak 12
# BERAPA sebanyak 13
# LAINNYA sebanyak 13

IDK-MRC Test memiliki banyak pertanyaan APA sekitar: 268
IDK-MRC Test memiliki banyak pertanyaan DIMANA sekitar: 49
IDK-MRC Test memiliki banyak pertanyaan KAPAN sekitar: 173
IDK-MRC Test memiliki banyak pertanyaan SIAPA sekitar: 76
IDK-MRC Test memiliki banyak pertanyaan KENAPA sekitar: 8
IDK-MRC Test memiliki banyak pertanyaan BAGAIMANA sekitar: 12
IDK-MRC Test memiliki banyak pertanyaan BERAPA sekitar: 109
IDK-MRC Test memiliki banyak pertanyaan LAINNYA sekitar: 149

IDK-MRC Test memiliki banyak pertanyaan APA sekitar: 31.753554502369667 %
IDK-MRC Test memiliki banyak pertanyaan DIMANA sekitar: 5.80568720379147 %
IDK-MRC Test memiliki banyak pertanyaan KAPAN sekitar: 20.497630331753555 %
IDK-MRC Test memiliki banyak pertanyaan SIAPA sekitar: 9.004739336492891 %
IDK-MRC Test memiliki banyak pertanyaan KENAPA sekitar: 0.9478672985781991 %
IDK-MRC Test memiliki banyak pertanyaan BAGAIMANA sekitar: 1.4218009478672986 %
IDK-MRC Test memiliki banyak pertanyaan BERAPA sekitar: 12.914691943

In [90]:
apa_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Apa', 'Apakah', 'apa', 'apakah'], number=14)
dimana_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Dimana', 'dimana', 'mana'], number=13)
kapan_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Kapan', 'kapan'], number=14)
siapa_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Siapa', 'siapa'], number=13)
kenapa_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Mengapa', 'Kenapa', 'mengapa', 'kenapa'], number=8)
bagaimana_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Bagaimana', 'bagaimana'], number=12)
berapa_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Berapa', 'Berapakah', 'berapa', 'berapakah'], number=13)
lainnya_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type="lainnya", number=14)

# Bagian TYDIQA-ID

In [27]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'tydiqa_id' in x.dataset_name)[0].load_dataset()

df_test = pd.DataFrame(data_qas_id['test'])

cols = ['context', 'question', 'answer']
new_df_test = pd.DataFrame(columns=cols)

for i in range(len(df_test['context'])):
    answer_start = df_test['context'][i].index(df_test['label'][i])
    answer_end = answer_start + len(df_test['label'][i])
    new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                        'question': df_test["question"][i], 
                                        'answer': df_test["label"][i]}, ignore_index=True)
   
test_dataset = Dataset.from_dict(new_df_test)
data_qas_id_tydiqa_id = DatasetDict({"test": test_dataset})
data_qas_id_tydiqa_id = pd.DataFrame(data_qas_id_tydiqa_id['test'])
data_qas_id_tydiqa_id

  0%|          | 0/3 [00:00<?, ?it/s]

,context,question,answer
0,Ernest Douwes Dekker wafat dini hari tanggal 2...,dimanakah Dr. Ernest François Eugène Douwes D...,28 Agustus 1950
1,"Pada tanggal 18 Februari 2008, desain Yoo Kerl...",Siapa arsitek Balai Kota Seoul?,Yoo Kerl
2,Sebagai tindak lanjut Atlantic Charter tersebu...,Kapan PBB mulai terbentuk ?,24 Oktober 1945
3,"Dia dipenjarakan di Puri Dragsholm, 75 kilomet...",Dimana James Hepburn meninggal?,Puri Dragsholm
4,"Lahir di Sheffield, South Yorkshire, Vardy mem...",Dimana Jamie Richard Vardy lahir?,"Sheffield, South Yorkshire"
...,...,...,...
850,Raden Patah (Jawa: ꦫꦢꦺꦤ꧀ꦦꦠꦃcode: jav promoted ...,Siapa raja Demak pertama ?,Raden Patah
851,Laut dalam adalah lapisan terbawah dari lautan...,Berapakah kedalaman laut yang disebut dengan l...,1828 m
852,"Justus Heurnius (lahir di Utrecht, Belanda, 15...",Kapan Justus Heurnius lahir ?,1587
853,Frekuensi suara atau frekuensi audio yaitu get...,Apakah yang dimaksud dengan frekuensi audio?,getaran frekuensi yang terdengar oleh manusia ...


In [42]:
question_type(data=data_qas_id_tydiqa_id, nama="TyDIQA-ID Test")

# Target: 100 anotasi -> 100/8 = 12.5
# APA sebanyak 15
# DIMANA sebanyak 14
# KAPAN sebanyak 15
# SIAPA sebanyak 15
# KENAPA sebanyak 6
# BAGAIMANA sebanyak 5
# BERAPA sebanyak 15
# LAINNYA sebanyak 15

TyDIQA-ID Test memiliki banyak pertanyaan APA sekitar: 301
TyDIQA-ID Test memiliki banyak pertanyaan DIMANA sekitar: 34
TyDIQA-ID Test memiliki banyak pertanyaan KAPAN sekitar: 136
TyDIQA-ID Test memiliki banyak pertanyaan SIAPA sekitar: 56
TyDIQA-ID Test memiliki banyak pertanyaan KENAPA sekitar: 6
TyDIQA-ID Test memiliki banyak pertanyaan BAGAIMANA sekitar: 5
TyDIQA-ID Test memiliki banyak pertanyaan BERAPA sekitar: 131
TyDIQA-ID Test memiliki banyak pertanyaan LAINNYA sekitar: 186

TyDIQA-ID Test memiliki banyak pertanyaan APA sekitar: 35.2046783625731 %
TyDIQA-ID Test memiliki banyak pertanyaan DIMANA sekitar: 3.976608187134503 %
TyDIQA-ID Test memiliki banyak pertanyaan KAPAN sekitar: 15.906432748538013 %
TyDIQA-ID Test memiliki banyak pertanyaan SIAPA sekitar: 6.549707602339182 %
TyDIQA-ID Test memiliki banyak pertanyaan KENAPA sekitar: 0.7017543859649122 %
TyDIQA-ID Test memiliki banyak pertanyaan BAGAIMANA sekitar: 0.5847953216374269 %
TyDIQA-ID Test memiliki banyak pertanyaan 

In [89]:
apa_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Apa', 'Apakah', 'apa', 'apakah'], number=15)
dimana_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Dimana', 'dimana', 'mana'], number=14)
kapan_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Kapan', 'kapan'], number=15)
siapa_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Siapa', 'siapa'], number=15)
kenapa_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Mengapa', 'Kenapa', 'mengapa', 'kenapa'], number=6)
bagaimana_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Bagaimana', 'bagaimana'], number=5)
berapa_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Berapa', 'Berapakah', 'berapa', 'berapakah'], number=15)
lainnya_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type="lainnya", number=15)

# Bagian SQUAD-ID

In [26]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'squad_id' in x.dataset_name)[0].load_dataset()

df_validation = pd.DataFrame(data_qas_id['validation'])

cols = ['context', 'question', 'answer']
new_df_val = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_validation['context']))):
    new_df_val = new_df_val.append({'context': df_validation["context"][i], 
                                    'question': df_validation["question"][i], 
                                    'answer': eval(df_validation["answer"][i][0])['text']},
                                ignore_index=True)

validation_dataset = Dataset.from_dict(new_df_val)

data_qas_id_squad_id = DatasetDict({"validation": validation_dataset})
data_qas_id_squad_id = pd.DataFrame(data_qas_id_squad_id['validation'])
data_qas_id_squad_id

  0%|          | 0/2 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████| 11873/11873 [00:28<00:00, 415.34it/s]


,context,question,answer
0,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Di negara apa Normandia berada?,Perancis
1,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Kapan Normandia di Normandia?,10 dan ke
2,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Dari negara mana asal Norse?,"Denmark, Islandia dan Norwegia"
3,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Siapa pemimpin Norse?,Rollo
4,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Abad berapa pertama kali Normandia mendapatkan...,abad ke-10
...,...,...,...
11868,"Gaya pon memiliki padanan metrik, yang lebih j...",Berapa satuan gaya yang jarang digunakan sama ...,sthène
11869,"Gaya pon memiliki padanan metrik, yang lebih j...",Apa yang tidak memiliki mitra metrik?,pound-force
11870,"Gaya pon memiliki padanan metrik, yang lebih j...",Berapakah gaya yang diberikan oleh gravitasi s...,kilogram-force
11871,"Gaya pon memiliki padanan metrik, yang lebih j...",Kekuatan apa yang mengarah ke satuan massa yan...,kilogram-force


In [40]:
question_type(data=data_qas_id_squad_id, nama="SQuAD-ID Test")

SQuAD-ID Test memiliki banyak pertanyaan APA sekitar: 6215
SQuAD-ID Test memiliki banyak pertanyaan DIMANA sekitar: 899
SQuAD-ID Test memiliki banyak pertanyaan KAPAN sekitar: 692
SQuAD-ID Test memiliki banyak pertanyaan SIAPA sekitar: 1030
SQuAD-ID Test memiliki banyak pertanyaan KENAPA sekitar: 191
SQuAD-ID Test memiliki banyak pertanyaan BAGAIMANA sekitar: 340
SQuAD-ID Test memiliki banyak pertanyaan BERAPA sekitar: 1470
SQuAD-ID Test memiliki banyak pertanyaan LAINNYA sekitar: 1036

SQuAD-ID Test memiliki banyak pertanyaan APA sekitar: 52.345658216120604 %
SQuAD-ID Test memiliki banyak pertanyaan DIMANA sekitar: 7.5718015665796345 %
SQuAD-ID Test memiliki banyak pertanyaan KAPAN sekitar: 5.82835003790112 %
SQuAD-ID Test memiliki banyak pertanyaan SIAPA sekitar: 8.67514528762739 %
SQuAD-ID Test memiliki banyak pertanyaan KENAPA sekitar: 1.6086919902299335 %
SQuAD-ID Test memiliki banyak pertanyaan BAGAIMANA sekitar: 2.863640192032342 %
SQuAD-ID Test memiliki banyak pertanyaan BERAPA

In [31]:
data_qas_id_idk_mrc.to_csv("idk_mrc.csv")
data_qas_id_tydiqa_id.to_csv("tydiqa_id.csv")
#data_qas_id_squad_id.to_excel("squad_id.xlsx")